In [ ]:
import sys
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy import ndimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# 1. LEGACY COMPATIBILITY FIXES
# ==========================================
import pandas.core.indexes as indexes
sys.modules['pandas.indexes'] = indexes

# ==========================================
# 2. DATA LOADING & PREPROCESSING
# ==========================================
def load_and_preprocess():
    file_path = 'LSWMD.pkl'
    if os.path.exists(file_path):
        print("Opening LSWMD.pkl using legacy byte-stream decoding...")
        with open(file_path, 'rb') as f:
            data = pickle.load(f, encoding='latin1')
        df = pd.DataFrame(data)
        
        # Clean labels
        df['failureType'] = df['failureType'].apply(
            lambda x: x[0][0] if isinstance(x, (list, np.ndarray)) and len(x) > 0 else 'none'
        )
        # Filter for actual defects
        df_defects = df[df['failureType'] != 'none'].copy()
        print(f"✅ Successfully loaded {len(df_defects)} defect samples.")
    else:
        # Fallback Test Data
        print("⚠️ File not found, generating synthetic data...")
        test_data = {'waferMap': [np.random.randint(0, 3, (26, 26)) for _ in range(500)],
                     'failureType': [np.random.choice(['Center', 'Donut', 'Scratch']) for _ in range(500)]}
        df_defects = pd.DataFrame(test_data)

    # Label Mapping
    label_map = {val: i for i, val in enumerate(df_defects['failureType'].unique())}
    df_defects['label_id'] = df_defects['failureType'].map(label_map)
    
    # Preprocessing Pipeline
    processed_images = []
    for x in df_defects['waferMap']:
        denoised = ndimage.median_filter(x, size=2)
        resized = tf.image.resize(denoised[:, :, np.newaxis], (28, 28)).numpy()
        processed_images.append(resized)
    
    X = np.array(processed_images).astype('float32') / 2.0
    y = tf.keras.utils.to_categorical(df_defects['label_id'])
    return X, y, label_map

# Run Loading
X, y, label_map = load_and_preprocess()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 3. THE EDGE-AI ARCHITECTURE
# ==========================================

def build_precision_model(num_classes):
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.SeparableConv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2), 
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = build_precision_model(len(label_map))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ==========================================
# 4. TRAINING & EXPORT
# ==========================================
print("\nStarting Training...")
# We save the 'history' object here so it is defined for the plots below
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Quantization for NXP eIQ
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('wafer_defect_edge.tflite', 'wb') as f:
    f.write(tflite_model)

# ==========================================
# 5. VISUALIZATION (RESULTS)
# ==========================================
def plot_results(history, model, X_test, y_test, label_map):
    # 1. Accuracy/Loss Curves
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(history.history['accuracy'], label='Train')
    ax1.plot(history.history['val_accuracy'], label='Val')
    ax1.set_title('Accuracy'); ax1.legend()
    
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Val')
    ax2.set_title('Loss'); ax2.legend()
    plt.show()

    # 2. Confusion Matrix
    
    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
                xticklabels=label_map.keys(), yticklabels=label_map.keys())
    plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.show()
    
    print("\n--- Numerical Precision Report ---")
    print(classification_report(y_true, y_pred, target_names=list(label_map.keys())))

# RUN VISUALS
plot_results(history, model, X_test, y_test, label_map)

Opening LSWMD.pkl using legacy byte-stream decoding...


C:\Users\ajith\AppData\Local\Temp\ipykernel_36652\35077386.py:28: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  data = pickle.load(f, encoding='latin1')


✅ Successfully loaded 25519 defect samples.

Starting Training...
Epoch 1/10
 319/1276 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.5225 - loss: 1.4166